<a href="https://colab.research.google.com/github/yugratna19/TrekNepal/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [175]:
# Install dependencies
!pip install -q kaggle

In [176]:
# Download the dataset
!kaggle datasets download -d bibekrai44/nepal-treking-dataset

# Unzip the dataset
!unzip nepal-treking-dataset.zip

In [177]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [178]:
df = pd.read_csv('/content/Trek Data.csv')

## **Data preprocessing**

In [179]:
df.head()

,Unnamed: 0,Trek,Cost,Time,Trip Grade,Max Altitude,Accomodation,Best Travel Time,Contact or Book your Trip
0,0,Everest Base Camp Trek,"\n$1,420 USD",16 Days,Moderate,5545 m,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com
1,1,Everest Base Camp Short Trek,"\n$1,295 USD",14 Days,Moderate,5545 m,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com
2,2,Everest Base Camp Heli Shuttle Trek,\n$2000 USD,12 Days,Moderate,5545 m,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com
3,3,Everest Base Camp Heli Trek,"\n$3,300 USD",11 Days,Moderate,5545 m,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com
4,4,Everest Base Camp Trek for Seniors,"\n$1,800 USD",20 Days,Moderate,5545 m,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com


In [180]:
df.describe()

,Unnamed: 0
count,383.00000
mean,191.00000
std,110.70682
min,0.00000
25%,95.50000
50%,191.00000
75%,286.50000
max,382.00000


In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 383 non-null    int64 
 1   Trek                       383 non-null    object
 2   Cost                       383 non-null    object
 3   Time                       383 non-null    object
 4   Trip Grade                 383 non-null    object
 5   Max Altitude               383 non-null    object
 6   Accomodation               383 non-null    object
 7   Best Travel Time           383 non-null    object
 8   Contact or Book your Trip  383 non-null    object
dtypes: int64(1), object(8)
memory usage: 27.1+ KB


In [182]:
df.isnull().sum()

Unnamed: 0                   0
Trek                         0
Cost                         0
Time                         0
Trip Grade                   0
Max Altitude                 0
Accomodation                 0
Best Travel Time             0
Contact or Book your Trip    0
dtype: int64

In [183]:
df = df.drop(columns=['Unnamed: 0'])

In [184]:
df.head()

,Trek,Cost,Time,Trip Grade,Max Altitude,Accomodation,Best Travel Time,Contact or Book your Trip
0,Everest Base Camp Trek,"\n$1,420 USD",16 Days,Moderate,5545 m,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com
1,Everest Base Camp Short Trek,"\n$1,295 USD",14 Days,Moderate,5545 m,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com
2,Everest Base Camp Heli Shuttle Trek,\n$2000 USD,12 Days,Moderate,5545 m,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com
3,Everest Base Camp Heli Trek,"\n$3,300 USD",11 Days,Moderate,5545 m,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com
4,Everest Base Camp Trek for Seniors,"\n$1,800 USD",20 Days,Moderate,5545 m,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com


In [185]:
# Convert 'Cost'column string to float
def convert_cost(cost_str):
  cost_str =  cost_str.lower().replace('$','').replace('usd','')
  cost_str =  cost_str.replace(',','')
  return float(cost_str)

# Convert 'Max'
def convert_max_altitude(altitude_str):
  altitude_str =  altitude_str.lower().replace('m', '')
  try:
    altitude_str =  altitude_str.lower().replace(',', '')
  except:
    altitude_str = altitude_str
  return int(altitude_str)

def convert_days(day_str):
  day_str =  day_str.lower().replace('days', '')
  return int(day_str)

In [186]:
df['Cost'].head()

0    \n$1,420     USD
1    \n$1,295     USD
2         \n$2000 USD
3    \n$3,300     USD
4    \n$1,800     USD
Name: Cost, dtype: object

In [187]:
df['Max Altitude'].unique()

array(['5545 m', '5360 m', '3900 m', '3860 m', '4210 m', '4230 m',
       '4200 m', '4,200 m', '5416 m', '2012m', '3210 m', '1990 m',
       '5106 m', '5220 m', '5050 m', '4600m', '2607 m', '3650 m',
       '4,130m', '5,545m', '5,416m', '3,870m', '6,340m', '5,357m',
       '3,540m', '5,160m', '5,000m', '3,055m', '5,546m', '5,143m',
       '6,000m', '1,730m', '4,870m', '5,320m', '4,460m', '4,500m',
       '3,210m', '5,122m', '1,550m', '3,860m', '4200m', '4,773m',
       '5,220m'], dtype=object)

In [188]:
df['Time'].unique()

array([' 16 Days', ' 14 Days', ' 12 Days', ' 11 Days', ' 20 Days',
       ' 19 Days', ' 22 Days', ' 7 Days', ' 10 Days', ' 9 Days',
       ' 21 Days', ' 8 Days', ' 18 Days', ' 17 Days', ' 5 Days',
       ' 15 Days', ' 18 days', ' 25 Days', ' 27 Days', ' 13 Days',
       '16 Days', '15 Days', '10 Days', '19 Days', '13 Days', '9 Days',
       '17 Days', '11 Days'], dtype=object)

In [189]:
df['Cost'] = df['Cost'].apply(convert_cost)

In [190]:
df['Max Altitude'] = df['Max Altitude'].apply(convert_max_altitude)

In [191]:
df['Time'] = df['Time'].apply(convert_days)

In [192]:
df.head()

,Trek,Cost,Time,Trip Grade,Max Altitude,Accomodation,Best Travel Time,Contact or Book your Trip
0,Everest Base Camp Trek,1420.0,16,Moderate,5545,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com
1,Everest Base Camp Short Trek,1295.0,14,Moderate,5545,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com
2,Everest Base Camp Heli Shuttle Trek,2000.0,12,Moderate,5545,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com
3,Everest Base Camp Heli Trek,3300.0,11,Moderate,5545,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com
4,Everest Base Camp Trek for Seniors,1800.0,20,Moderate,5545,Hotel/Guesthouse,March - May & Sept - Dec,https://www.nepalhikingteam.com


In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Trek                       383 non-null    object 
 1   Cost                       383 non-null    float64
 2   Time                       383 non-null    int64  
 3   Trip Grade                 383 non-null    object 
 4   Max Altitude               383 non-null    int64  
 5   Accomodation               383 non-null    object 
 6   Best Travel Time           383 non-null    object 
 7   Contact or Book your Trip  383 non-null    object 
dtypes: float64(1), int64(2), object(5)
memory usage: 24.1+ KB
